# Get Tickers From Firm Names

Use prior data to try to get the tickers of the firms.

Should use credit rating data, can use sector data additionally if needed.

In [12]:
# Packages
import pandas as pd

## Load Reference Data

In [13]:
# Load credit rating datasets
# ~\Box\STAT 222 Capstone\Raw Data\Supplementary Credit Rating Data From Kaggle\corporateCreditRatingWithFinancialRatios.csv
# ~\Box\STAT 222 Capstone\Raw Data\Credit Rating Data From Kaggle\corporate_rating.csv
ccrwfr = pd.read_csv('~/Box/STAT 222 Capstone/Raw Data/Supplementary Credit Rating Data From Kaggle/corporateCreditRatingWithFinancialRatios.csv')
print(ccrwfr.head())
corporate_rating = pd.read_csv('~/Box/STAT 222 Capstone/Raw Data/Credit Rating Data From Kaggle/corporate_rating.csv')
print(corporate_rating.head())

                        Rating Agency                     Corporation Rating  \
0  Standard & Poor's Ratings Services       American States Water Co.     A-   
1  Standard & Poor's Ratings Services  Automatic Data Processing Inc.    AAA   
2  Standard & Poor's Ratings Services                      Avnet Inc.   BBB-   
3  Standard & Poor's Ratings Services    California Water Service Co.    AA-   
4  Standard & Poor's Ratings Services            Cardinal Health Inc.      A   

  Rating Date      CIK  Binary Rating  SIC Code Sector Ticker  Current Ratio  \
0  2010-07-30  1056903              1    4941.0  Utils    AWR         1.1507   
1  2010-09-16     8670              1    7374.0  BusEq    ADP         1.1129   
2  2010-11-23     8858              1    5065.0  Shops    AVT         1.9276   
3  2010-06-29  1035201              1    4941.0  Utils    CWT         0.8358   
4  2010-07-14   721371              1    5122.0  Shops    CAH         1.2931   

   ...  EBITDA Margin  Pre-Tax Profit 

In [14]:
# Keep where Rating Agency == "Standard & Poor's Ratings Services" and "Rating Agency Name" == "Standard & Poor's Ratings Services" respectively
s_and_p_ccrwfr = ccrwfr[ccrwfr['Rating Agency'] == "Standard & Poor's Ratings Services"]
s_and_p_corporate_rating = corporate_rating[corporate_rating['Rating Agency Name'] == "Standard & Poor's Ratings Services"]
# Keep columns Ticker and Corporation and Symbol and Name respectively
# Rename to Ticker and Corporation
s_and_p_ccrwfr = s_and_p_ccrwfr[['Ticker', 'Corporation']]
s_and_p_corporate_rating = s_and_p_corporate_rating[['Symbol', 'Name']].rename(columns={'Symbol': 'Ticker', 'Name': 'Corporation'})
# Stack the two datasets
s_and_p_ticker_names = pd.concat([s_and_p_ccrwfr, s_and_p_corporate_rating], axis=0)
# Drop duplicates
s_and_p_ticker_names = s_and_p_ticker_names.drop_duplicates()
# Uppercase Corporation
s_and_p_ticker_names['Corporation'] = s_and_p_ticker_names['Corporation'].str.upper()
# Iterate over and print all rows
for index, row in s_and_p_ticker_names.iterrows():
    print(row['Ticker'], row['Corporation'])

AWR AMERICAN STATES WATER CO.
ADP AUTOMATIC DATA PROCESSING INC.
AVT AVNET INC.
CWT CALIFORNIA WATER SERVICE CO.
CAH CARDINAL HEALTH INC.
CRS CARPENTER TECHNOLOGY CORP.
SBS COMPANHIA DE SANEAMENTO BASICO DO ESTADO DE SAO PAULO
COMP COMPASS GROUP PLC
DEO DIAGEO PLC
ETR ENTERGY CORP
FTI FMC TECHNOLOGIES INC.
FDX FEDEX CORP.
FLS FLOWSERVE CORP.
FLR FLUOR CORP.
FDP FRESH DEL MONTE PRODUCE INC.
GIS GENERAL MILLS INC.
JBL JABIL INC.
KMT KENNAMETAL INC.
KSS KOHL'S CORP.
KOP KOPPERS HOLDINGS INC.
KEP KOREA ELECTRIC POWER CORP.
MGEE MADISON GAS & ELECTRIC CO.
MCHP MICRON TECHNOLOGY INC.
MSFT MICROSOFT CORP.
NGG NATIONAL GRID GAS PLC
NGG NATIONAL GRID USA
NEM NEWMONT USA LTD.
NWN NORTHWEST NATURAL GAS CO.
PNM PNM RESOURCES INC
RCII RENT-A-CENTER INC.
SJR SHAW COMMUNICATIONS INC.
SCCO SOUTHERN COPPER CORP.
SCCO SOUTHERN POWER COMPANY
TECK TECK RESOURCES LTD.
TAC TRANSALTA CORP.
TPC TUTOR PERINI CORPORATION
UHS UNIVERSAL HEALTH SERVICES INC.
VGR VECTOR LTD.
ALE ALLETE INC.
ABT ABBOTT LABORATORIES


In [ ]:
# Cleaning
s_and_p_ticker_names_clean = s_and_p_ticker_names.copy()
# For column Corporation:
# Strip "CO."
# Strip "INC."
# Strip "PLC"
# Strip "CORP."
# Strip "CORP"
# Strip "LTD."
# Strip "COMPANY"
# Strip "CORPORATION"
# Strip "S.A."
# Strip "LP"
# Strip "LLC"
# Strip "INCORPORATED"
# Strip "INTERNATIONAL"
# Strip "PJSC"
# Strip "N.V."
# Strip "L.P."
# Strip "S.A.B. DE C.V."
# Strip "S.A. DE C.V."
# Strip "S.A.A."
# Strip "GROUP HOLDING LIMITED"
# Strip "& CO."
# Strip ", INC. (THE)"
# Strip "RESORT PROPERTIES FINANCE INC."
# Strip "OPERATING CO., LLC"
# Strip "AG & CO. KGAA"
# Replace "SAP SE" with "SAP"
# Strip "B.V."
# Strip " ULC"
# Strip "GMBH"
# Replace "NOVARTIS AG" with "NOVARTIS"
# Strip "S.A.R.L."

# Strip "INC" at the end of strings

# Strip leading and trailing whitespace

# Strip any punctuation on the end of strings

# Iterate over and print all rows
for index, row in s_and_p_ticker_names_clean.iterrows():
    print(row['Ticker'], row['Corporation'])